In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.graph_objs as go 
import plotly.offline as py 
import math

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import VotingClassifier
from sklearn.base import clone 
from sklearn.metrics import ConfusionMatrixDisplay
import xgboost as xgb

In [3]:
import os
os.chdir("C:\\uit\\elliptic\\Elliptic_Dataset")
print("\nTransaction features: \n")
df_txs_features = pd.read_csv("txs_features.csv")
print(df_txs_features)

print("\nTransaction classes: \n")
df_txs_classes = pd.read_csv("txs_classes.csv")
print(df_txs_classes)

print("\nTransaction-Transaction edgelist: \n")
df_txs_edgelist = pd.read_csv("txs_edgelist.csv")

print(df_txs_edgelist)


Transaction features: 

             txId  Time step  Local_feature_1  Local_feature_2  \
0            3321          1        -0.169615        -0.184668   
1           11108          1        -0.137586        -0.184668   
2           51816          1        -0.170103        -0.184668   
3           68869          1        -0.114267        -0.184668   
4           89273          1         5.202107        -0.210553   
...           ...        ...              ...              ...   
203764  158304003         49        -0.165622        -0.139563   
203765  158303998         49        -0.167040        -0.139563   
203766  158303966         49        -0.167040        -0.139563   
203767  161526077         49        -0.172212        -0.139573   
203768  194103537         49        -0.172212        -0.139573   

        Local_feature_3  Local_feature_4  Local_feature_5  Local_feature_6  \
0             -1.201369        -0.121970        -0.043875        -0.113002   
1             -1.201369   

In [4]:
print("\ntxs_features.csv for txId = 272145560\n")
print(df_txs_features[df_txs_features['txId']==272145560])

print("\ntxs_classes.csv for txId = 272145560\n")
print(df_txs_classes[df_txs_classes['txId']==272145560])

print("\ntxs_edgelist.csv for txId = 272145560\n")
df_txs_edgelist[(df_txs_edgelist['txId1']==272145560) | (df_txs_edgelist['txId2']==272145560)]


txs_features.csv for txId = 272145560

             txId  Time step  Local_feature_1  Local_feature_2  \
105573  272145560         24        -0.155493        -0.107012   

        Local_feature_3  Local_feature_4  Local_feature_5  Local_feature_6  \
105573        -1.201369         -0.12197        -0.043875        -0.113002   

        Local_feature_7  Local_feature_8  ...  in_BTC_min  in_BTC_max  \
105573        -0.061584        -0.145749  ...      2.7732      2.7732   

        in_BTC_mean  in_BTC_median  in_BTC_total  out_BTC_min  out_BTC_max  \
105573       2.7732         2.7732        2.7732     0.001917     2.770883   

        out_BTC_mean  out_BTC_median  out_BTC_total  
105573        1.3864          1.3864         2.7728  

[1 rows x 184 columns]

txs_classes.csv for txId = 272145560

             txId  class
105573  272145560      1

txs_edgelist.csv for txId = 272145560



,txId1,txId2
123072,272145560,296926618
123272,272145560,272145556
125873,299475624,272145560


In [5]:
df_txs_features = df_txs_features.merge(df_txs_classes, left_on = "txId" , right_on = "txId" , how = "left")
# print(txs.shape)

print(df_txs_features["class"].value_counts())

class
3    157205
2     42019
1      4545
Name: count, dtype: int64


In [6]:
df_txs_features = df_txs_features.dropna()
df_txs_features

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.534072,0.534072,0.534072,0.534072,1.668990e-01,0.367074,0.266986,0.266986,0.533972,3
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,5.611878,5.611878,5.611878,5.611878,5.861940e-01,5.025584,2.805889,2.805889,5.611778,3
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.456608,0.456608,0.456608,0.456608,2.279902e-01,0.228518,0.228254,0.228254,0.456508,3
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,8.000000,3.102967,1.000000,9.308900,1.229000e+00,8.079800,4.654400,4.654400,9.308800,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,852.164680,852.164680,852.164680,852.164680,1.300000e-07,41.264036,0.065016,0.000441,852.164680,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,115.952889,115.952889,115.952889,115.952889,1.653300e+00,114.299544,57.976422,57.976422,115.952844,3
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,114.250098,114.250098,114.250098,114.250098,2.035300e-02,114.229700,57.125027,57.125027,114.250053,3
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,113.606771,113.606771,113.606771,113.606771,9.257490e-01,112.680977,56.803363,56.803363,113.606726,3
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,112.680977,112.680977,112.680977,112.680977,3.026970e-01,112.378235,56.340466,56.340466,112.680932,3


In [7]:
for column in df_txs_features.columns[168:]:
    if column != "class":
        feature = np.array(df_txs_features[column]).reshape(-1,1)
        scaler = MinMaxScaler()
        scaler.fit(feature)
        feature_scaled = scaler.transform(feature)
        df_txs_features[column] = feature_scaled.reshape(1,-1)[0]

In [8]:
# remove 'unknown' transactions
data = df_txs_features["txId"] #.loc[(df_txs_features['class'] != 3), 'txId']
df_txs_features_selected = df_txs_features.loc[df_txs_features['txId'].isin(data)]
print(df_txs_features_selected["class"].value_counts())

class
3    156759
2     41500
1      4545
Name: count, dtype: int64


In [9]:
# Goal: binary classification of 0,1
# 0: licit, 1: illicit

X_data = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] < 35) & (df_txs_features_selected['class'] != 3), 'txId']
X_training_timesteps = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data)]
X_train = X_training_timesteps.drop(columns=['txId', 'class', 'Time step'])

X_data_test = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] >= 35), 'txId']
X_testing_timesteps = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data_test)]
X_test = X_testing_timesteps.drop(columns=['txId', 'class', 'Time step'])

X_data_test_class01 = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] >= 35) & (df_txs_features_selected['class'] != 3), 'txId']
X_testing_timesteps_class01 = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data_test_class01)]
X_test_class01 = X_testing_timesteps_class01.drop(columns=['txId', 'class', 'Time step'])

y_training_timesteps = X_training_timesteps[['class']]
y_training_timesteps = y_training_timesteps['class'].apply(lambda x: x-1 ) # change illicit (class-2) to '0' for classification
y_train = y_training_timesteps

y_testing_timesteps = X_testing_timesteps[['class']]
y_testing_timesteps = y_testing_timesteps['class'].apply(lambda x: x-1 ) # change illicit (class-2) to '0' for classification
y_test = y_testing_timesteps

y_testing_timesteps_class01 = X_testing_timesteps_class01[['class']]
y_testing_timesteps_class01 = y_testing_timesteps_class01['class'].apply(lambda x: x-1 ) # change illicit (class-2) to '0' for classification
y_test_class01 = y_testing_timesteps_class01

In [10]:
y_train.value_counts()

class
1    26237
0     3462
Name: count, dtype: int64

In [11]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_SMOTE, y_train_SMOTE = smote.fit_resample(X_train, y_train)


In [12]:
# 0) Chuẩn bị TRAIN graph (gồm cả unknown=3) & TEST graph (chỉ 0/1)
# =========================
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from sklearn.metrics import precision_recall_fscore_support, f1_score, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42
torch.manual_seed(SEED); np.random.seed(SEED)

# Map label: 1->0, 2->1, 3->-1 (unknown)
def map_label(x):
    return {1:0, 2:1}.get(x, -1)

# ---- Train graph: timesteps < 35, GIỮ cả class==3 (unlabeled) ----
train_nodes_all = df_txs_features_selected[df_txs_features_selected['Time step'] < 35].copy()
train_nodes_all['label'] = train_nodes_all['class'].apply(map_label)
feat_cols = [c for c in df_txs_features_selected.columns if c not in ['txId','class','Time step']]

X_train_all = torch.tensor(train_nodes_all[feat_cols].values, dtype=torch.float)
y_train_all = torch.tensor(train_nodes_all['label'].values, dtype=torch.long)
txids_train = torch.tensor(train_nodes_all['txId'].astype(int).values, dtype=torch.long)

def build_edge_index(df_edges, valid_nodes_df):
    node_ids = valid_nodes_df['txId'].astype(int).values
    id2idx = {tid:i for i,tid in enumerate(node_ids)}
    edges = df_edges[(df_edges['txId1'].isin(id2idx)) & (df_edges['txId2'].isin(id2idx))].copy()
    if len(edges) > 0:
        edge_index = torch.tensor(
            edges[['txId1','txId2']].replace(id2idx).values.T, dtype=torch.long
        )
    else:
        edge_index = torch.empty((2,0), dtype=torch.long)
    return edge_index

edge_index_train = build_edge_index(df_txs_edgelist, train_nodes_all)

train_data = Data(x=X_train_all, edge_index=edge_index_train, y=y_train_all)
train_data.node_ids = txids_train
train_mask_init = (y_train_all >= 0)  # True cho node có nhãn 0/1
unlabeled_mask_init = (y_train_all < 0)
# ---- Test graph: timesteps >= 35, chỉ 0/1 ----
test_nodes = df_txs_features_selected[
    (df_txs_features_selected['Time step'] >= 35) & (df_txs_features_selected['class'] != 3)
].copy()
test_nodes['label'] = test_nodes['class'].apply(lambda x: x-1)

X_test = torch.tensor(test_nodes[feat_cols].values, dtype=torch.float)
y_test = torch.tensor(test_nodes['label'].values, dtype=torch.long)
txids_test = torch.tensor(test_nodes['txId'].astype(int).values, dtype=torch.long)

edge_index_test = build_edge_index(df_txs_edgelist, test_nodes)
test_data = Data(x=X_test, edge_index=edge_index_test, y=y_test)
test_data.node_ids = txids_test


In [13]:
# =========================
# 1) Model GraphSAGE
# =========================
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hid_dim=128, out_dim=2, dropout=0.5):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hid_dim)
        self.conv2 = SAGEConv(hid_dim, hid_dim)
        self.head  = nn.Linear(hid_dim, out_dim)
        self.dropout = dropout
    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index); h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)
        h = self.conv2(h, edge_index); h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)
        return self.head(h)

def class_weights_from_mask(y, mask):
    y_lab = y[mask].cpu().numpy()
    c0 = (y_lab==0).sum(); c1 = (y_lab==1).sum()
    w = torch.tensor([1.0/(c0+1e-8), 1.0/(c1+1e-8)], dtype=torch.float)
    return w

In [14]:
@torch.no_grad()
def evaluate_df(model, data, device, name="TEST"):
    model.eval()
    logits = model(data.x.to(device), data.edge_index.to(device))
    preds = logits.argmax(dim=1).cpu().numpy()
    y_true = data.y.cpu().numpy()
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, preds, zero_division=0, labels=[0,1])
    micro = f1_score(y_true, preds, average='micro', zero_division=0)
    cm = confusion_matrix(y_true, preds, labels=[0,1])
    print(f"\n[{name}] Class0: P={prec[0]:.3f} R={rec[0]:.3f} F1={f1[0]:.3f} | "
          f"Class1: P={prec[1]:.3f} R={rec[1]:.3f} F1={f1[1]:.3f} | Micro={micro:.3f}")
    print("CM:\n", cm)
    # Trả DF để tiện ensemble nếu cần
    return pd.DataFrame({"txId": data.node_ids.cpu().numpy(),
                         f"pred_{name.lower()}": preds})

def train_epochs(model, data, mask_bool, epochs=50, lr=3e-3, wd=5e-5, dropout=0.5):
    import torch.nn.functional as F
    model.train()
    model.dropout = dropout

    opt  = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    sched= torch.optim.lr_scheduler.StepLR(opt, step_size=50, gamma=0.5)

    # Tính class weight từ LABELED NODES (mask_bool) — ở CPU là được
    w = class_weights_from_mask(data.y, mask_bool).to(device)
    crit = nn.CrossEntropyLoss(weight=w)

    # Lấy chỉ số của node có nhãn rồi CHUYỂN LÊN GPU
    idx = mask_bool.nonzero(as_tuple=False).view(-1).to(device)  # long on device

    # (tuỳ chọn) kiểm tra kiểu dữ liệu
    # assert idx.dtype == torch.long

    for ep in range(1, epochs+1):
        opt.zero_grad()

        # Đưa dữ liệu lên cùng device
        x  = data.x.to(device)
        ei = data.edge_index.to(device)
        y  = data.y.to(device)        # <<< QUAN TRỌNG: y lên GPU

        out = model(x, ei)            # out ở GPU

        # Index-select trên CÙNG DEVICE
        out_l = out.index_select(0, idx)
        y_l   = y.index_select(0, idx)

        loss = crit(out_l, y_l)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
        opt.step()
        sched.step()

        if ep % 10 == 0:
            print(f"  - ep {ep:03d} | loss={loss.item():.4f}")


@torch.no_grad()
def pseudo_label_once(model, data, unlabeled_mask, tau=0.95, max_add_per_class=None):
    """
    Chọn pseudo-label cho node unlabeled với max prob >= tau.
    max_add_per_class: giới hạn số lượng thêm cho mỗi lớp (int) để tránh lệch lớp; None nếu không giới hạn.
    Trả về: idx_add (tensor cpu), y_add (tensor cpu với 0/1)
    """
    model.eval()
    logits = model(data.x.to(device), data.edge_index.to(device))
    prob = F.softmax(logits, dim=1).cpu()
    unl_idx = unlabeled_mask.nonzero(as_tuple=False).view(-1)
    if len(unl_idx) == 0:
        return torch.empty(0, dtype=torch.long), torch.empty(0, dtype=torch.long)

    prob_unl = prob.index_select(0, unl_idx)
    conf, y_hat = prob_unl.max(dim=1)  # [num_unl], [num_unl]
    sel = (conf >= tau)

    if sel.sum().item() == 0:
        return torch.empty(0, dtype=torch.long), torch.empty(0, dtype=torch.long)

    idx_candidates = unl_idx[sel]
    y_candidates = y_hat[sel]

    # Giới hạn mỗi lớp (optional)
    if isinstance(max_add_per_class, int):
        idx_keep = []
        for cls in [0,1]:
            ids_cls = idx_candidates[(y_candidates==cls)]
            take = ids_cls[:max_add_per_class]  # lấy đầu tiên theo thứ tự conf (đang chưa sort)
            idx_keep.append(take)
        idx_keep = torch.cat(idx_keep) if len(idx_keep) else torch.empty(0, dtype=torch.long)
        mask_keep = torch.isin(idx_candidates, idx_keep)
        idx_sel = idx_candidates[mask_keep]
        y_sel = y_candidates[mask_keep]
    else:
        idx_sel = idx_candidates
        y_sel = y_candidates

    return idx_sel.cpu(), y_sel.cpu()


In [15]:
# =========================
# 2) Self-training loop
# =========================
in_dim = train_data.x.size(1)
model = GraphSAGE(in_dim=in_dim, hid_dim=128, out_dim=2, dropout=0.5).to(device)

# Masks khởi tạo
labeled_mask = train_mask_init.clone()        # bool
unlabeled_mask = unlabeled_mask_init.clone()  # bool
y_train_work = train_data.y.clone()           # chứa -1 cho unlabeled

# (Tuỳ chọn) Tách một validation window cuối (vd t=30..34) để dừng sớm -> bỏ qua cho gọn

R = 3                 # số vòng self-training
E_init = 300           # epoch train ban đầu
E_iter = 80           # epoch train mỗi vòng
tau = 0.95            # ngưỡng confidence
decay = 0.0           # nếu muốn giảm ngưỡng mỗi vòng, đặt ví dụ 0.02

print("=== Phase 0: supervised trên labeled nodes ban đầu ===")
train_epochs(model, train_data, labeled_mask, epochs=E_init, lr=3e-3, wd=5e-5, dropout=0.5)

for r in range(1, R+1):
    print(f"\n=== Self-training round {r} (tau={tau:.2f}) ===")
    # Chọn pseudo-label từ UNLABELED trong TRAIN graph
    idx_add, y_add = pseudo_label_once(model, train_data, unlabeled_mask, tau=tau, max_add_per_class=None)
    print(f"  + chọn {len(idx_add)} pseudo-labeled (0/1)")

    if len(idx_add) == 0:
        print("  + không có node đủ confidence — dừng sớm")
        break

    # Gán nhãn & mở rộng mask
    y_train_work[idx_add] = y_add
    labeled_mask[idx_add] = True
    unlabeled_mask[idx_add] = False
    train_data.y = y_train_work  # cập nhật vào data

    # Huấn luyện thêm với mask mới
    train_epochs(model, train_data, labeled_mask, epochs=E_iter, lr=3e-3, wd=5e-5, dropout=0.5)

    # (tuỳ chọn) giảm ngưỡng
    tau = max(0.80, tau - decay)

=== Phase 0: supervised trên labeled nodes ban đầu ===
  - ep 010 | loss=0.3350
  - ep 020 | loss=0.2398
  - ep 030 | loss=0.1896
  - ep 040 | loss=0.1548
  - ep 050 | loss=0.1323
  - ep 060 | loss=0.1201
  - ep 070 | loss=0.1124
  - ep 080 | loss=0.0991
  - ep 090 | loss=0.0929
  - ep 100 | loss=0.0902
  - ep 110 | loss=0.0847
  - ep 120 | loss=0.0809
  - ep 130 | loss=0.0795
  - ep 140 | loss=0.0749
  - ep 150 | loss=0.0700
  - ep 160 | loss=0.0706
  - ep 170 | loss=0.0689
  - ep 180 | loss=0.0685
  - ep 190 | loss=0.0673
  - ep 200 | loss=0.0680
  - ep 210 | loss=0.0637
  - ep 220 | loss=0.0630
  - ep 230 | loss=0.0627
  - ep 240 | loss=0.0656
  - ep 250 | loss=0.0652
  - ep 260 | loss=0.0627
  - ep 270 | loss=0.0614
  - ep 280 | loss=0.0612
  - ep 290 | loss=0.0617
  - ep 300 | loss=0.0587

=== Self-training round 1 (tau=0.95) ===
  + chọn 75876 pseudo-labeled (0/1)
  - ep 010 | loss=0.0529
  - ep 020 | loss=0.0446
  - ep 030 | loss=0.0357
  - ep 040 | loss=0.0339
  - ep 050 | loss

In [16]:
# =========================
# 3) Đánh giá
# =========================
_ = evaluate_df(model, test_data, device, name="TEST")


[TEST] Class0: P=0.579 R=0.684 F1=0.627 | Class1: P=0.977 R=0.965 F1=0.971 | Micro=0.946
CM:
 [[  741   342]
 [  538 14725]]


In [17]:
# ================================
# GraphSAGE + Self-training (only pseudo-label class 0) on grouped graph
# ================================
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from sklearn.metrics import precision_recall_fscore_support, f1_score, confusion_matrix

# ----------------
# 0) Config & Utils
# ----------------
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TRAIN_TS = list(range(1, 35))   # < 35
TEST_TS  = list(range(35, 50))  # >= 35

# Label mapping: 1->0 (licit), 2->1 (illicit), 3->-1 (unknown)
def map_label(x):
    return {1:0, 2:1}.get(x, -1)

# Chọn cột feature
feature_cols = [c for c in df_txs_features_selected.columns if c not in ['txId','class','Time step']]

def build_edge_index(df_edges, valid_nodes_df, undirected=True):
    """Lọc edge theo tập node; trả về edge_index (LongTensor [2, E])"""
    node_ids = valid_nodes_df['txId'].astype(int).values
    id2idx = {tid:i for i, tid in enumerate(node_ids)}
    if 'timestep' in df_edges.columns:
        # không nối train/test; caller tự lọc theo timestep ở ngoài nếu cần
        edges = df_edges[df_edges['txId1'].isin(id2idx) & df_edges['txId2'].isin(id2idx)].copy()
    else:
        edges = df_edges[df_edges['txId1'].isin(id2idx) & df_edges['txId2'].isin(id2idx)].copy()

    if len(edges) > 0:
        src = edges['txId1'].astype(int).map(id2idx).values.tolist()
        dst = edges['txId2'].astype(int).map(id2idx).values.tolist()
        if undirected:
            edge_index = torch.tensor([src + dst, dst + src], dtype=torch.long)
        else:
            edge_index = torch.tensor([src, dst], dtype=torch.long)
    else:
        edge_index = torch.empty((2,0), dtype=torch.long)
    return edge_index


In [18]:
# ----------------
# 1) Build TRAIN (gộp timesteps < 35, GIỮ cả class=3) & TEST (>=35, chỉ 0/1)
# ----------------
train_nodes_all = df_txs_features_selected[df_txs_features_selected['Time step'] < 35].copy()
train_nodes_all['y_mapped'] = train_nodes_all['class'].apply(map_label)

X_train_all = torch.tensor(train_nodes_all[feature_cols].values, dtype=torch.float)
y_train_all = torch.tensor(train_nodes_all['y_mapped'].values, dtype=torch.long)
train_txids = torch.tensor(train_nodes_all['txId'].astype(int).values, dtype=torch.long)

edge_index_train = build_edge_index(df_txs_edgelist, train_nodes_all, undirected=True)

train_data = Data(x=X_train_all, edge_index=edge_index_train, y=y_train_all)
train_data.node_ids = train_txids
labeled_mask   = (y_train_all >= 0)   # 0/1
unlabeled_mask = (y_train_all <  0)   # -1

# TEST: chỉ lấy class 1/2
test_nodes = df_txs_features_selected[
    (df_txs_features_selected['Time step'] >= 35) & (df_txs_features_selected['class'] != 3)
].copy()
test_nodes['y_mapped'] = test_nodes['class'].apply(lambda x: x-1)
X_test = torch.tensor(test_nodes[feature_cols].values, dtype=torch.float)
y_test = torch.tensor(test_nodes['y_mapped'].values, dtype=torch.long)
test_txids = torch.tensor(test_nodes['txId'].astype(int).values, dtype=torch.long)
edge_index_test = build_edge_index(df_txs_edgelist, test_nodes, undirected=True)
test_data = Data(x=X_test, edge_index=edge_index_test, y=y_test)
test_data.node_ids = test_txids

In [19]:
# ----------------
# 2) Model: GraphSAGE
# ----------------
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hid_dim=128, out_dim=2, dropout=0.5):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hid_dim)
        self.conv2 = SAGEConv(hid_dim, hid_dim)
        self.head  = nn.Linear(hid_dim, out_dim)
        self.dropout = dropout
    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index); h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)
        h = self.conv2(h, edge_index); h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)
        return self.head(h)

In [20]:
# ----------------
# 3) Helper: class weights, train stage, pick unlabeled pred=0
# ----------------
def class_weights_from_mask(y, mask_bool):
    y_lab = y[mask_bool].cpu().numpy()
    c0 = (y_lab==0).sum(); c1 = (y_lab==1).sum()
    w0 = 1.0/(c0 + 1e-8); w1 = 1.0/(c1 + 1e-8)
    return torch.tensor([w0, w1], dtype=torch.float)

def train_one_stage(model, data, labeled_mask, device, epochs=60, lr=3e-3, wd=5e-5, dropout=0.5):
    model.train(); model.dropout = dropout
    opt  = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    sched= torch.optim.lr_scheduler.StepLR(opt, step_size=50, gamma=0.5)
    w = class_weights_from_mask(data.y, labeled_mask).to(device)
    crit = nn.CrossEntropyLoss(weight=w)

    for ep in range(1, epochs+1):
        opt.zero_grad()
        x, ei = data.x.to(device), data.edge_index.to(device)
        y = data.y.to(device); m = labeled_mask.to(device)
        out = model(x, ei)
        loss = crit(out[m], y[m])
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 2.0)
        opt.step(); sched.step()
        if ep % 10 == 0:
            print(f"  - ep {ep:03d} | loss={loss.item():.4f}")

@torch.no_grad()
def pick_unlabeled_pred0(model, data, unlabeled_mask, tau_neg=0.92, margin=0.10, device=None, max_add=None):
    """
    Chỉ chọn unlabeled có dự đoán lớp 0 với p0>=tau_neg và |p1-p0|>=margin.
    Trả về: idx_chosen (LongTensor cpu), labels (toàn 0)
    """
    model.eval()
    x, ei = data.x.to(device), data.edge_index.to(device)
    logits = model(x, ei)
    prob = F.softmax(logits, dim=1).detach().cpu().numpy()
    p0, p1 = prob[:,0], prob[:,1]
    pred = (p1 >= p0).astype(int)
    margin_ok = (np.abs(p1 - p0) >= margin)

    unl_idx = unlabeled_mask.nonzero(as_tuple=False).view(-1).cpu().numpy()
    if unl_idx.size == 0:
        return torch.empty(0, dtype=torch.long), torch.empty(0, dtype=torch.long)

    cand_local = np.where((pred[unl_idx] == 0) & (p0[unl_idx] >= tau_neg) & (margin_ok[unl_idx]))[0]
    if cand_local.size == 0:
        return torch.empty(0, dtype=torch.long), torch.empty(0, dtype=torch.long)

    # ưu tiên theo p0 giảm dần
    order = np.argsort(-p0[unl_idx[cand_local]])
    cand_local = cand_local[order]

    if (max_add is not None) and (cand_local.size > max_add):
        cand_local = cand_local[:max_add]

    chosen_global = unl_idx[cand_local]
    labels = np.zeros(len(chosen_global), dtype=np.int64)
    return torch.tensor(chosen_global, dtype=torch.long), torch.tensor(labels, dtype=torch.long)

In [21]:
# ----------------
# 4) Self-training loop: chỉ bơm pseudo=0
# ----------------
def self_training_pred0_only(model, data, labeled_mask, unlabeled_mask,
                             device,
                             rounds=5, tau_neg=0.92, margin=0.10,
                             decay=0.00, max_add_per_round=None,
                             stage0_epochs=200, stageK_epochs=80):
    print("=== Phase 0: supervised on labeled ===")
    train_one_stage(model, data, labeled_mask, device, epochs=stage0_epochs, lr=3e-3, wd=5e-5, dropout=0.5)

    cur_tau = tau_neg
    for r in range(1, rounds+1):
        print(f"\n=== Round {r} (tau_neg={cur_tau:.2f}, margin={margin:.2f}) ===")
        idx_add, y_add = pick_unlabeled_pred0(model, data, unlabeled_mask,
                                              tau_neg=cur_tau, margin=margin,
                                              device=device, max_add=max_add_per_round)
        n_add = idx_add.numel()
        print(f"  + will add: {n_add} pseudo negatives")
        if n_add == 0:
            print("  + no confident negatives → stop")
            break

        # gán nhãn & cập nhật mask
        data.y[idx_add] = y_add.to(data.y.dtype)
        labeled_mask[idx_add]   = True
        unlabeled_mask[idx_add] = False

        # train lại
        train_one_stage(model, data, labeled_mask, device, epochs=stageK_epochs, lr=3e-3, wd=5e-5, dropout=0.5)

        # curriculum: nới ngưỡng nếu muốn
        cur_tau = max(0.80, cur_tau - decay)

In [22]:
# ----------------
# 5) Train + Self-training + Eval
# ----------------
model = GraphSAGE(in_dim=train_data.x.size(1), hid_dim=128, out_dim=2, dropout=0.5).to(device)

# y trong data: 0/1 cho labeled, -1 cho unlabeled
train_data.y = train_data.y.clone()  # đã là y_train_all theo map_label

# chạy self-training (chỉ bơm class 0 từ unlabeled)
self_training_pred0_only(
    model, train_data,
    labeled_mask=labeled_mask.clone(),
    unlabeled_mask=unlabeled_mask.clone(),
    device=device,
    rounds=5,           # tăng nếu muốn
    tau_neg=0.92,       # hạ xuống 0.90 nếu ít mẫu được thêm
    margin=0.10,        # tăng lên 0.15 nếu muốn an toàn hơn
    decay=0.00,         # ví dụ 0.02 để giảm dần ngưỡng mỗi vòng
    max_add_per_round=20000,  # hoặc None
    stage0_epochs=200,
    stageK_epochs=80
)

# Đánh giá trên TEST
@torch.no_grad()
def evaluate(model, data, device, name="TEST"):
    model.eval()
    out = model(data.x.to(device), data.edge_index.to(device))
    preds = out.argmax(dim=1).cpu().numpy()
    y_true = data.y.cpu().numpy()
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, preds, zero_division=0, labels=[0,1])
    micro = f1_score(y_true, preds, average='micro', zero_division=0)
    cm = confusion_matrix(y_true, preds, labels=[0,1])
    print(f"\n[{name}] Class0: P={prec[0]:.3f} R={rec[0]:.3f} F1={f1[0]:.3f} | "
          f"Class1: P={prec[1]:.3f} R={rec[1]:.3f} F1={f1[1]:.3f} | Micro={micro:.3f}")
    print("Confusion Matrix:\n", cm)

evaluate(model, test_data, device, name="TEST")

=== Phase 0: supervised on labeled ===
  - ep 010 | loss=0.3026
  - ep 020 | loss=0.2166
  - ep 030 | loss=0.1678
  - ep 040 | loss=0.1323
  - ep 050 | loss=0.1142
  - ep 060 | loss=0.1026
  - ep 070 | loss=0.0883
  - ep 080 | loss=0.0809
  - ep 090 | loss=0.0758
  - ep 100 | loss=0.0699
  - ep 110 | loss=0.0662
  - ep 120 | loss=0.0619
  - ep 130 | loss=0.0620
  - ep 140 | loss=0.0563
  - ep 150 | loss=0.0524
  - ep 160 | loss=0.0526
  - ep 170 | loss=0.0498
  - ep 180 | loss=0.0525
  - ep 190 | loss=0.0461
  - ep 200 | loss=0.0469

=== Round 1 (tau_neg=0.92, margin=0.10) ===
  + will add: 7438 pseudo negatives
  - ep 010 | loss=0.0609
  - ep 020 | loss=0.0525
  - ep 030 | loss=0.0449
  - ep 040 | loss=0.0380
  - ep 050 | loss=0.0370
  - ep 060 | loss=0.0349
  - ep 070 | loss=0.0334
  - ep 080 | loss=0.0314

=== Round 2 (tau_neg=0.92, margin=0.10) ===
  + will add: 4449 pseudo negatives
  - ep 010 | loss=0.0521
  - ep 020 | loss=0.0430
  - ep 030 | loss=0.0348
  - ep 040 | loss=0.0364